In [1]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)

from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()

#hugging face token 입력


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: read).
Your token has been saved to /home/20223149/.cache/huggingface/token
Login successful


In [2]:
huggingface_dataset_name = "beomi/KoAlpaca-v1.1a"
#data set load
dataset = load_dataset(huggingface_dataset_name)

In [3]:
dataset['train'][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [5]:
len(dataset['train'])

21155

In [6]:
compute_dtype = getattr(torch, "float16")
#모델을 4bit 형식으로 로드(메모리 소비가 줄어듦)
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [7]:
model_name='microsoft/phi-2'
device_map = {"": 0}
#모델을 양자화하여 다운(or load)
original_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/gatoai/python/venv/3.10/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
#hugging face의 transformers 라이브로리를 사용하여 자연어 처리 모델에 대한 토크나이저를 설정하는 함수
#AutoTokenizer.from_pretrained: 사전 훈련된 토크나이저를 불러옵니다.
#model_name: 사용할 사전 훈련된 모델의 이름 또는 경로를 지정합니다.
#trust_remote_code=True: 원격 코드를 신뢰하도록 지정합니다. 이것은 Hugging Face 모델 허브에서 토크나이저를 다운로드할 때 사용됩니다.
#padding_side="left": 패딩을 왼쪽에 추가합니다. 이는 토크나이저가 입력 시퀀스를 패딩할 때 어느 쪽에 패딩을 추가할지를 지정합니다.
#add_eos_token=True: End-Of-Sequence (EOS) 토큰을 추가합니다. 이는 문장의 끝을 나타내는 특수 토큰입니다.
#add_bos_token=True: Begin-Of-Sequence (BOS) 토큰을 추가합니다. 이는 문장의 시작을 나타내는 특수 토큰입니다.
#use_fast=False: 빠른 토크나이저를 사용하지 않도록 설정합니다. 이것은 더 빠른 토크나이저가 아닌 표준 토크나이저를 사용하도록 강제합니다.
tokenizer_ko = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b",trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
#tokenizer.pad_token = tokenizer.eos_token: 이 부분은 패딩 토큰을 EOS (End-Of-Sequence) 토큰으로 설정합니다. 이렇게 하면 모델이 패딩을 식별하는 데 사용되는 토큰을 EOS 토큰으로 사용하게 됩니다.
tokenizer_ko.pad_token = tokenizer_ko.eos_token

#이 코드는 주어진 모델의 토크나이저를 설정하고, EOS 및 BOS 토큰을 추가하며, 패딩을 왼쪽에 추가하도록 구성합니다.

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
#hugging face의 transformers 라이브로리를 사용하여 자연어 처리 모델에 대한 토크나이저를 설정하는 함수
#AutoTokenizer.from_pretrained: 사전 훈련된 토크나이저를 불러옵니다.
#model_name: 사용할 사전 훈련된 모델의 이름 또는 경로를 지정합니다.
#trust_remote_code=True: 원격 코드를 신뢰하도록 지정합니다. 이것은 Hugging Face 모델 허브에서 토크나이저를 다운로드할 때 사용됩니다.
#padding_side="left": 패딩을 왼쪽에 추가합니다. 이는 토크나이저가 입력 시퀀스를 패딩할 때 어느 쪽에 패딩을 추가할지를 지정합니다.
#add_eos_token=True: End-Of-Sequence (EOS) 토큰을 추가합니다. 이는 문장의 끝을 나타내는 특수 토큰입니다.
#add_bos_token=True: Begin-Of-Sequence (BOS) 토큰을 추가합니다. 이는 문장의 시작을 나타내는 특수 토큰입니다.
#use_fast=False: 빠른 토크나이저를 사용하지 않도록 설정합니다. 이것은 더 빠른 토크나이저가 아닌 표준 토크나이저를 사용하도록 강제합니다.
tokenizer_phi_2 = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
#tokenizer.pad_token = tokenizer.eos_token: 이 부분은 패딩 토큰을 EOS (End-Of-Sequence) 토큰으로 설정합니다. 이렇게 하면 모델이 패딩을 식별하는 데 사용되는 토큰을 EOS 토큰으로 사용하게 됩니다.
tokenizer_phi_2.pad_token = tokenizer_phi_2.eos_token

#이 코드는 주어진 모델의 토크나이저를 설정하고, EOS 및 BOS 토큰을 추가하며, 패딩을 왼쪽에 추가하도록 구성합니다.

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def gen_ko(model_name, prompt, max_length):
    # 입력 텍스트를 토크나이즈하고 모델에 입력할 형식으로 변환
    input_ids = tokenizer_ko.encode(prompt, return_tensors="pt", truncation=True)
    
    # 모델에 입력하여 텍스트 생성
    outputs = model_name.generate(input_ids, max_length=max_length)

    # print(len(outputs))
    # print(outputs)
    
    
    # 생성된 텍스트 디코딩
    generated_text = tokenizer_ko.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

def gen_phi_2(model_name, prompt, max_length):
    # 입력 텍스트를 토크나이즈하고 모델에 입력할 형식으로 변환
    input_ids = tokenizer_phi_2.encode(prompt, return_tensors="pt", truncation=True)
    
    # 모델에 입력하여 텍스트 생성
    outputs = model_name.generate(input_ids, max_length=max_length)

    # print(len(outputs))
    # print(outputs)
    
    
    # 생성된 텍스트 디코딩
    generated_text = tokenizer_phi_2.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

In [11]:
%%time
# 실행 시간을 알려주는 주피터 매직코드
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10

## test를 위한 데이터
prompt = dataset['train'][index]['instruction']
output_data = dataset['train'][index]['output']

# prompt 포맷
formatted_prompt = f"Instruction:\n{prompt}\nOutput:\n"

# model 돌리기
res = gen_ko(original_model, formatted_prompt, 250)

# print('test 출력')
# 결과 출력
print(res)
# output만 출력
output = res.split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
# 입력 promt 출력
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
# 사람이 요약한 내용 출력
print(f'BASELINE HUMAN SUMMARY:\n{output_data}\n')
print(dash_line)
# 모델이 요약한 내용 출력
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

## 위 모델은 요약은 하지 않지만 필수 정보들은 모두 추출하는 것을 보여주고 있음
## 모델 fine-tuning 가능성 제시

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/gatoai/python/venv/3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Instruction:
왜 초밥에는 와사비가 들어갈까요? 
초밥은 안매운 음식인데, 와사비 때문에 너무 매울 때가 있습니다. 그래서 제가 궁금한 건 초밥에 와사비가 들어가는 이유가 무엇인가요?
Output:
왜 초밥에는 와사비가 들어가는 중가가*=6 키�약�하 약 일�딩 � 9 대� 국방 자본�� 커지��� 늦MC 자본*가운데� 들어간질 �E 스�총브라 뜻*�강�산 피( � ��산(�산� 어뵜질 사#X^W=)V 수성])vX^# 낫 28질�* 장영 정$(�%버린$ 낫 28%고마6대부E 접근 들 분리 접근 들이 접근 들 장비 긋=6 키�약�* 장영 정 전북 대� 내렸6가운데� 들어간질 �E 스�총브라 뜻*�강�산 피( � ��산(�
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
왜 초밥에는 와사비가 들어갈까요? 
초밥은 안매운 음식인데, 와사비 때문에 너무 매울 때가 있습니다. 그래서 제가 궁금한 건 초밥에 와사비가 들어가는 이유가 무엇인가요?
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
와사비는 기생충을 퇴치하기 위해 사용됩니다. 
초밥에 와사비를 함께 제공하는 이유는 그것이 기생충을 퇴치하기 위함입니다. 대부분의 날생선에는 기생충이 내장에 숨어있기 때문에, 그것을 제거하기 위해 고추냉이의 살균력을 활용하는 것입니다. 또한, 생강에도 살균 작용이 있어 식중독을 예방하는 역할을 합니다. 초밥집에서 얇게 쓴 생강을 제공하거나, 전갱이나 가다랭이 등에 생강을 곁들여 먹게 되는 것도 이와 같은 이유 때문입니다. 따라서, 와사비가 초밥과 함께 제공되는 것은 단순한

In [12]:
%%time
# 실행 시간을 알려주는 주피터 매직코드
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10

## test를 위한 데이터
prompt = dataset['train'][index]['instruction']
output_data = dataset['train'][index]['output']

# prompt 포맷
formatted_prompt = f"Instruction:\n{prompt}\nOutput:\n"

# model 돌리기
res = gen_phi_2(original_model, formatted_prompt, 250)

# print('test 출력')
# 결과 출력
print(res)
# output만 출력
output = res.split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
# 입력 promt 출력
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
# 사람이 요약한 내용 출력
print(f'BASELINE HUMAN SUMMARY:\n{output_data}\n')
print(dash_line)
# 모델이 요약한 내용 출력
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

## 위 모델은 요약은 하지 않지만 필수 정보들은 모두 추출하는 것을 보여주고 있음
## 모델 fine-tuning 가능성 제시

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruction:
왜 초밥에는 와사비가 들어갈까요? 
초밥은 안매운 음식인데, 와사비 때문에 너무 매울 때가 있습니다. 그래서 제가 궁금한 건 초밥에 와사비가 들어가는 이유가 무엇인가요?
Output:
와사비가 들어가는 이유가 무�
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
왜 초밥에는 와사비가 들어갈까요? 
초밥은 안매운 음식인데, 와사비 때문에 너무 매울 때가 있습니다. 그래서 제가 궁금한 건 초밥에 와사비가 들어가는 이유가 무엇인가요?
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
와사비는 기생충을 퇴치하기 위해 사용됩니다. 
초밥에 와사비를 함께 제공하는 이유는 그것이 기생충을 퇴치하기 위함입니다. 대부분의 날생선에는 기생충이 내장에 숨어있기 때문에, 그것을 제거하기 위해 고추냉이의 살균력을 활용하는 것입니다. 또한, 생강에도 살균 작용이 있어 식중독을 예방하는 역할을 합니다. 초밥집에서 얇게 쓴 생강을 제공하거나, 전갱이나 가다랭이 등에 생강을 곁들여 먹게 되는 것도 이와 같은 이유 때문입니다. 따라서, 와사비가 초밥과 함께 제공되는 것은 단순한 맛을 위한 것 뿐만 아니라, 위생적인 목적도 있기 때문입니다.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
와사비가 들어가는 이유가 무�
CPU times: user 1.97 s, sys: 67.5 ms, total: 2.03 s
Wall time:

In [13]:
# 위에서 다운 받은 데이터는 fine-tuning에 직접 사용될 수 없다
# 모델이 이해할 수 있는 방식으로 프롬프트의 형식을 지정해서 전처리를 해야된다
# HuggingFace 모델 문서를 참조하면 아래 지정된 형식의 대화 및 요약을 사용하여 프롬프트를 생성해야 한다
# 대화-요약(프롬프트-응답) 쌍을 LLM에 대한 명시적 지침으로 변환해야 한다
# text라는 애를 만듦
# 입력을 프롬프트 형식으로 변환함
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters 
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Answer the questions below"
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"
    
    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['instruction']}" if sample["instruction"] else None
    response = f"{RESPONSE_KEY}\n{sample['output']}"
    end = f"{END_KEY}"
    
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

print(dataset['train'][0])
print('---------------------------')
print(create_prompt_formats(dataset['train'][0]))
print('---------------------------')
print(create_prompt_formats(dataset['train'][0])['instruction'])
print('---------------------------')
print(create_prompt_formats(dataset['train'][0])['text'])

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?', 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}
---------------------------
{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?', 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=

In [14]:
## prompt를 모델 토크나이저를 사용하여 토큰화된 프롬프트로 처리
## 여기서의 목표는 일관된 길이의 입력 시퀀스를 생성하는 것
## 일관된 길이의 입력 시퀀스는 효율성을 최적화하고 계산 오버헤드를 최소화하여 언어 모델을 미세 조정하는 데 도움이 된다.
## 이러한 시퀀스가 모델의 최대 토큰 제한을 초과하지 않도록 확인하는 것이 중요하다

from functools import partial

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
# 모델의 구성을 통해 최대 길이 설정을 가져오는 함수
# 모델의 구성 중에서 "n_positions", "max_position_embeddings", "seq_length"와 같은 설정을 확인하여 최대 길이를 찾습니다.
# 만약 최대 길이 설정이 없는 경우 기본값으로 1024를 사용합니다.
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length
# "n_positions":
    #이 설정은 모델이 처리할 수 있는 최대 위치 임베딩의 개수를 나타냅니다.
    #위치 임베딩은 입력 토큰의 위치 정보를 인코딩하는 데 사용됩니다.
    #따라서 이 값은 모델이 처리할 수 있는 최대 입력 시퀀스의 길이를 제한하는 데 사용됩니다.
# "max_position_embeddings":
    #이 설정은 모델이 처리할 수 있는 최대 입력 시퀀스의 길이를 나타냅니다.
    #즉, 입력 시퀀스의 최대 길이가 이 값보다 작거나 같아야 합니다.
    #이 값은 모델이 학습될 때 정의되며, 모델의 아키텍처에 따라 다를 수 있습니다.
#"seq_length":
    #이 설정은 모델이 입력으로 처리할 수 있는 최대 시퀀스의 길이를 나타냅니다.
    #입력 시퀀스의 최대 길이가 이 값보다 작거나 같아야 합니다.
    #이 값은 모델의 아키텍처에 따라 다르며, 주로 특정 언어 모델링 작업에 적합한 값을 선택하여 사용합니다.
#이러한 설정들은 모델이 처리할 수 있는 입력의 형태와 길이를 제한하고, 모델의 효율성을 최적화하는 데 중요한 역할을 합니다.
#이 값들은 모델을 초기화할 때 설정되며, 모델을 학습하는 동안 변경되지 않는다


# 주어진 배치를 토크나이징하여 전처리하는 함수입니다.
# 각 텍스트를 토크나이즈하고 최대 길이에 맞게 잘라내는 작업을 수행합니다.
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
#데이터셋을 전처리하여 모델 학습에 사용할 수 있는 형식으로 준비하는 함수입니다.
    #먼저 각 샘플에 프롬프트를 추가합니다.
    #다음으로 preprocess_batch 함수를 적용하여 각 배치를 전처리합니다. 이때 remove_columns 매개변수를 통해 불필요한 열을 제거합니다.
    #입력 시퀀스의 길이가 최대 길이를 초과하는 샘플을 제거합니다.
    #마지막으로 데이터셋을 섞습니다.
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    print(type(dataset))
    
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)
    
    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    # partial: 함수를 편하게 만듦
    # 여기서 데이터셋을 토큰화함
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    # 이러면 dataset에 text만 남음
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['url'],
    )

    # Filter out samples that have input_ids exceeding max_length
    # 입력 시퀀스의 길이가 max_length보다 큰 샘플을 필터링하여 제거
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    
    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [15]:
## Pre-process dataset
max_length = get_max_length(original_model)
print(max_length)

print(dataset['train'][0])

train_dataset = preprocess_dataset(tokenizer_ko, max_length,seed, dataset)
# eval_dataset = preprocess_dataset(tokenizer_ko, max_length,seed, dataset)

print(train_dataset['train'][0])

Found max lenth: 2048
2048
{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?', 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}
<class 'datasets.dataset_dict.DatasetDict'>
Preprocessing dataset...


Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

Filter:   0%|          | 0/21155 [00:00<?, ? examples/s]

{'instruction': '케첩과 마요네즈의 입구 모양은 왜 다른가요? \n보통 케찹과 마요네즈는 한 세트자나요? 그런데 보면 케찹은 원터치 뚜껑에다가 새 케찹은 뚜껑을 돌려서 열면 얇은 알미늄 같은 막도 이꾸또 케찹이 나오는 구멍이 동그랗자나요? 근데 마요네즈는 돌리는 뚜껑에다가 처음에 새로 사도 알미늄 같은 막두 엄꾸나오는 구멍 모양도 별 모양인데 왜 그런가요?', 'output': '케첩과 마요네즈의 입구 모양이 다른 것은 끈적이는 정도 즉, 물질의 점성(Viscosity) 차이 때문입니다. 마요네즈는 케첩보다 점성이 높기 때문에 돌리는 뚜껑에 별 모양의 입구를 만들어도 내부 액체의 모양이 유지됩니다. 하지만 케첩은 끈적거림이 적기 때문에 별 모양의 입구를 만들어도 케첩 내부의 액체가 곧 둥글게 뭉쳐지기 때문에 케첩의 뚜껑은 동그란 구멍입니다. 따라서 입구 모양은 물질의 점성(Viscosity) 차이 때문입니다.', 'text': '\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruct: Answer the questions below\n\n케첩과 마요네즈의 입구 모양은 왜 다른가요? \n보통 케찹과 마요네즈는 한 세트자나요? 그런데 보면 케찹은 원터치 뚜껑에다가 새 케찹은 뚜껑을 돌려서 열면 얇은 알미늄 같은 막도 이꾸또 케찹이 나오는 구멍이 동그랗자나요? 근데 마요네즈는 돌리는 뚜껑에다가 처음에 새로 사도 알미늄 같은 막두 엄꾸나오는 구멍 모양도 별 모양인데 왜 그런가요?\n\n### Output:\n케첩과 마요네즈의 입구 모양이 다른 것은 끈적이는 정도 즉, 물질의 점성(Viscosity) 차이 때문입니다. 마요네즈는 케첩보다 점성이 높기 때문에 돌리는 뚜껑에 별 모양의 입구를 만들어도 내부 액체의 모양이 유지됩니다. 하지만 케첩은 끈적거림이 적기 때문에 별 모양의 입구를 만들어도 

In [16]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


# 2 - Using the prepare_model_for_kbit_training method from PEFT
# Preparing the Model for QLoRA
original_model = prepare_model_for_kbit_training(original_model)

In [17]:
config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

peft_model = get_peft_model(original_model, config)

In [18]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


print(print_number_of_trainable_model_parameters(peft_model))



trainable model parameters: 20971520
all model parameters: 1542364160
percentage of trainable model parameters: 1.36%


In [19]:
output_dir = f'./peft-ko-training-{str(int(time.time()))}'
import transformers

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset['train'],
    eval_dataset=train_dataset['train'],
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer_ko, mlm=False),
)

/home/20223149/.local/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
peft_trainer.train()


/home/20223149/alpha_project/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
25,8.780600,8.934643
50,9.267200,8.934643
75,8.832400,8.934643
100,9.248000,8.934643
125,8.692300,8.934643
150,9.228900,8.934643
175,8.730200,8.934643
200,9.223400,8.934643
225,8.772600,8.934643
250,9.276800,8.934643


/home/20223149/alpha_project/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/20223149/alpha_project/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/20223149/alpha_project/.venv/lib/pyt

In [ ]:
print(1)